## Chains in Langchain

- LLMChain
- Sequential Chain 
    - SimpleSequentialChain
    - SequentialChain
- Router Chain

chain = (LLM + prompt_template + output parser) block - you can combine multiple blocks to carry out sequence of operations

In [1]:
# import environment 
from dotenv import load_dotenv,find_dotenv

_ = load_dotenv(find_dotenv())


In [48]:
import pandas as pd

from IPython.display import display

review_df = pd.read_csv("review_data.csv")
display(review_df)

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...
5,L'Or Espresso Café \n,Je trouve le goût médiocre. La mousse ne tient...
6,Hervidor de Agua Eléctrico,"Está lu bonita calienta muy rápido, es muy fun..."


## LLM Chain example

In [34]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

chat_model = ChatOpenAI(temperature=0.9)

In [29]:
product_prompt = ChatPromptTemplate.from_template("""
Generate an appropriate name for a company that makes```{product}```
""")

chain = LLMChain(llm=chat_model,prompt=product_prompt)

product_name = chain.run(product = review_df['Product'].iloc[0])

print(product_name)


Royal Comfort


## SimpleSequential Chain :

combine two chains using `+` operator , output of one chain is input for next

works well when subchain expect one input and give out one output


In [36]:
from langchain.chains import SimpleSequentialChain

prompt_one = ChatPromptTemplate.from_template("Generate one name to describe a company that makes `{product}`")

prompt_two = ChatPromptTemplate.from_template("Write a 20 word description for the Company name delimited by \
                                              ``` , Company : ```{company_name}```")

chain_one = LLMChain(llm = chat_model,prompt=prompt_one)
chain_two = LLMChain(llm = chat_model, prompt = prompt_two)

simple_seq_chain = SimpleSequentialChain(chains= [chain_one , chain_two], verbose=True)
print(review_df['Product'].iloc[2])

simple_seq_chain.run(input = review_df['Product'].iloc[2]) # need to use `input` , can't use other names

Luxury Air Mattress


> Entering new SimpleSequentialChain chain...
OpulentRest
OpulentRest is a luxurious company offering premium bedding products and accessories for a truly restful and opulent sleep experience.

> Finished chain.


'OpulentRest is a luxurious company offering premium bedding products and accessories for a truly restful and opulent sleep experience.'

## Sequential Chain

- Use `output_key` to retrieve output from chain , use it in different prompts

In [43]:
from langchain.chains import SequentialChain

first_prompt = ChatPromptTemplate.from_template("""Translate the text into English, text \n:{review}""")

#chain1: input -> review text , output English review
chain1 = LLMChain(llm=chat_model , prompt=first_prompt , output_key="eng_review")


In [44]:
# get the summary of eng_review
second_prompt = ChatPromptTemplate.from_template("""Summarize following reiview in 1 sentence:\n {eng_review}""")

chain2 = LLMChain(prompt=second_prompt , llm = chat_model , output_key="summary")


In [45]:
third_prompt = ChatPromptTemplate.from_template("""Identify the language of the following review : {review}""")

chain3 = LLMChain(llm = chat_model , prompt = third_prompt , output_key = "lang")

In [46]:
fourth_prompt = ChatPromptTemplate.from_template("""
    Write a one line follow up response to the summary :\n {summary} \n in following language \n : {lang}
""")
                                                 
chain4 = LLMChain(llm = chat_model , prompt=fourth_prompt , output_key= "followup")

In [47]:
# combine all 4 chains 
overall_chain = SequentialChain(chains=[chain1,chain2,chain3,chain4], input_variables=["review"],
                                output_variables=["eng_review","summary","lang","followup"],verbose=True)

In [ ]:
# now run using data
review = review_df.Review[6]
overall_chain(inputs = {"review":review}) # calls API for each LLMChain , so RateLimit error will be reached

In above example : 

{review} -> chain1 & chain3 
chain1 -> chain2-> chain4 , chain3-> chain4 

finally , chain4 -> followup